In [55]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import requests
import calendar
import geopandas as gpd
import os.path as os
import scipy.stats
import seaborn.palettes
import seaborn.utils
import sys
from census import Census
# from us import states
import http.client, urllib.request, urllib.parse, urllib.error, base64
#import config
import quickstart

In [56]:
from pydrive.auth import GoogleAuth

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # Creates local webserver and auto handles authentication.

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=570719947456-53haph7mjc317oel1ujh0kmjbfd499jf.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [57]:
# files in clean data google drive
from pydrive.drive import GoogleDrive
drive = GoogleDrive(gauth)
fileList = drive.ListFile({'q': "'1kiYbPtSYavHQKzyMWtXaLhP-JM2toXRm' in parents and trashed=false"}).GetList()
for file in fileList:
  print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "To Share"):
      fileID = file['id']

Title: weighted_borobox.csv, ID: 1H8yM7UH14YdMqi5ASjHMd_dwA4C7-lKe
Title: acs_2019_fuel_mobility.csv, ID: 1w3TaiypzzVSIYD9VvhCqwXaNfzkpj9V1
Title: acs_2009_2019.csv, ID: 17PvjIPesTzugausfzMi7rTiXahTiHaH1
Title: acs_2019only.csv, ID: 1z3-YL8loRQf9Qi6qSQUSwF8Wnwo-YQ5d
Title: pluto_df.csv, ID: 1j2KR6Tpa4CTA0k1omVfE52UYlMBMz17o
Title: hpd_agg_final.csv, ID: 1nIQD_4GDpg5ldTGgE5XqfiD11mxjrvEH
Title: ct_box_xwalk.csv, ID: 1nW-ODlhdUFKhTYzHZQOu3CCZN87oYXel
Title: fire_box_sum.csv, ID: 1zYSWlWtgNWsruaEkSSCqyXHIIdq7BEAP
Title: census_acs, ID: 1DjJ3pOLAo4MOpgmeNPSXJbRzF_S10fDM
Title: hpd_per_bbl.csv, ID: 1xXilfI1zEmyEYE_TtMdawcnmnZlTv__Z
Title: hpd_2012_2022_v3.csv, ID: 1HXst8E2ZuPVKvKe_9jmSQwKYppOJjraD
Title: pluto_borobox.csv, ID: 1Yf4xIIOBmKU2WIlImat8qaN3z2L239Nw
Title: fire_dispatch_tracts.csv, ID: 1psGrpBmGmNHPwM4lsRyiDX9tX5cf_UsO
Title: pluto_borobox.geojson, ID: 1UGelHNCzfSwsvbqBzyBX5GkrY_PmPGdp
Title: fire_dispatch.csv, ID: 1ulJlwpaezzq_aYV6jZzRWpmXLVlpO8qu
Title: dev_permits.csv, ID: 1mH

In [58]:
census_tract = drive.CreateFile({'id': '1z3-YL8loRQf9Qi6qSQUSwF8Wnwo-YQ5d'})
census_tract.GetContentFile('census_tract.csv')  

census_tract= pd.read_csv('census_tract.csv').drop("Unnamed: 0", axis = 1)
census_tract.head()

,geofips,totpop,popdens,areasqmile,males,females,pop18under,pop18_34,pop35_64,pop65plus,...,pcthotherf,pcthnofuel,pctsamehouse,pctmvdsamecounty,pctmvdsdiffcounty,pctmvdsdiffstate,pctmvdabroad,pop1yrplusinhh,pctpoprenterhh,geofips_recode
0,36005000100,6864,10599.550,0.647574,6345,519,103,3322,3392,47,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,2000100
1,36005000200,4532,24481.190,0.185122,2193,2339,1134,862,1605,931,...,0.00000,0.000000,97.917631,0.601573,0.555298,0.925497,0.000000,4515,0.297231,2000200
2,36005000400,5516,18537.140,0.297565,2810,2706,1071,1319,2362,764,...,0.00000,0.685602,100.000000,0.000000,0.000000,0.000000,0.000000,5450,0.374679,2000400
3,36005001600,5825,31101.490,0.187290,2422,3403,1546,1399,1978,902,...,0.00000,4.211060,96.144227,2.021676,1.042101,0.270946,0.521050,5561,0.354433,2001600
4,36005001900,3141,4877.152,0.644023,1583,1558,898,1052,1111,80,...,4.07785,3.058387,88.834951,3.353928,5.251545,2.250662,0.308914,2655,0.406403,2001900


In [59]:
# READ CENSUS DATA
#census_tract= pd.read_csv('acs_2019onlynew.csv')
#census_tract.head().columns

In [60]:
# census_tract= pd.read_csv('acs_2019only.csv').drop("Unnamed: 0", axis = 1).rename({"BoroCT2010":"boroct2010"}, axis = 1)

# labels_to_drop = ['pct18under',
#        'pct18_34', 'pct35_64', 'pct65plus', 'pcthispanic', 'pctnhwhite',
#        'pctnhblack', 'pctnhaian', 'pctnhasian', 'pctnhhawainpi', 'pctnhother',
#        'pctnhtwoplus', 'pctownerocch', 'pctrentocch']
# census_tract = census_tract.drop(labels_to_drop, axis = 1)
# # census_tract = census_tract[census_tract['year'] == 2019]
# census_tract = census_tract.fillna(0)

# # make region key the same as crosswalk file
# def get_region(x):
#     y = int(str(x)[:5])
#     z = str(x)[5:]
#     match y:
#         case (36005):
#             return int("2" + z)
#         case (36047):
#             return int("3" + z)
#         case (36061):
#             return int("1" + z)
#         case (36081):
#             return int("4" + z)
#         case (36085):
#             return int("5" + z)
# census_tract["geofips"] = census_tract["geofips"].apply(lambda x : get_region(x))

# census_tract

In [61]:
# census_tract_fuel = pd.read_csv('acs_2019_fuel_mobility.csv').drop(["Unnamed: 0"], axis=1)
# census_tract_fuel["Geo_FIPS"] = census_tract_fuel["Geo_FIPS"].apply(lambda x : get_region(x))
# census_tract_fuel.iloc[:,1:] = census_tract_fuel.iloc[:,1:].apply(lambda x : (x/100)*)
# census_tract_fuel.fillna(0)

In [62]:
crosswalk = drive.CreateFile({'id': '1nW-ODlhdUFKhTYzHZQOu3CCZN87oYXel'})
crosswalk.GetContentFile('crosswalk.csv')  

crosswalk= pd.read_csv('crosswalk.csv').drop("Unnamed: 0", axis = 1)
crosswalk.head()

,boroct2010,borobox,count,total,wgt
0,3070201,B4116,1614,1876,0.860341
1,3070201,B3994,228,1876,0.121535
2,3070201,B4103,34,1876,0.018124
3,3033600,B3520,836,913,0.915663
4,3033600,B3522,77,913,0.084337


In [63]:
#crosswalk = pd.read_csv('ct_box_xwalk1.csv')
#crosswalk = crosswalk.sort_values(by="borobox")

In [64]:
columnNames = ['borobox', 'count', 'totpop', 'popdens', 'areasqmile', 'males', 'females',
       'pop18under', 'pop18_34', 'pop35_64', 'pop65plus', 'pophispanic',
       'popnhwhite', 'popnhblack', 'popnhaian', 'popnhasian', 'popnhhawainpi',
       'popnhother', 'popnhtwoplus', 'housing', 'housingoccupied',
       'renterhousing', 'oc_halfless', 'oc_half_1', 'oc_1_onehalf',
       'oc_onehalf_2', 'oc_2plus', 'roc_halfless', 'roc_half_1',
       'roc_1_onehalf', 'roc_onehalf_2', 'roc_2plus', 'medianyearbuilt',
       'mediangrossrent', 'medianhhincome', 'gini', 'year', 'crowdingrenter', 'crowding', "housingoccupied2",
       "hhfgas","hhfelectricity","hhfoilkerosene","hhfcoalwood","hhfsolarenergy","hhfotherfuel","hhfnofuel"
       ,"pop1yrplusinhh","poprenterhu","popsamehouse","popmvdsamecounty","popmvddiffcounty","popmvdstate","popmvdabroad"]
weighted_borobox = pd.DataFrame(columns = columnNames)
numRuns = 0

for index, row in crosswalk.iterrows():
    box = row["borobox"]
    weight = row["wgt"]
    try:
        tract = int(row["boroct2010"])
    except:
        continue

    try: 
        
        row_census = census_tract.loc[census_tract['geofips_recode'] == tract].iloc[-1]

        # ------------ in order to test with smaller datset, uncomment the below three lines ---------
        # numRuns += 1
        # if numRuns > 10:
        #     break
        # ------------ end test section ---------------

# ---------add new row with borobox specified if row does not exist, otherwise edit existing row
        try:            # find existing row
            row_weighted = weighted_borobox.loc[weighted_borobox["borobox"] == box] # make copy of existing data to act on it
            row_weighted.at[0,'count'] = row_weighted.at[0,'count'] + 1
            for i, column in enumerate(columnNames):
                if i<2:
                    continue
                # elif (i<32):
                #     row_weighted.at[0,column] = (row_weighted.at[0,column] + (row_census[column] * weight))
                else:
                    # row_weighted.at[0,column] = row_weighted.at[0,column] + row_census[column]
                    row_weighted.at[0,column] = (row_weighted.at[0,column] + (row_census[column] * weight))
            weighted_borobox.loc[weighted_borobox["borobox"] == box] = row_weighted # paste in finalized new data

        except:         # create new row
            # create a column that initiates at one, if column is accessed again, increment. add up some columns that require averages. at end, divide to find target numbers.
            row_weighted = pd.DataFrame(columns = columnNames)
            row_weighted.at[0,'count'] = 1
            row_weighted.at[0,'borobox'] = box
            for i, column in enumerate(columnNames):
                if i<2:
                    continue
                # elif (i<32):
                #     row_weighted.at[0,column] = (row_census[column] * weight)
                else:
                    # row_weighted.at[0,column] = row_census[column]
                    row_weighted.at[0,column] = (row_census[column] * weight)
            weighted_borobox = pd.concat([weighted_borobox,row_weighted])
    except:
        print("error number: ")
        continue

In [65]:
for index, row in weighted_borobox.iterrows():
    instances = int(row["count"])
    row.at['medianyearbuilt'] = (row.at['medianyearbuilt']/instances)
    row.at['mediangrossrent'] = (row.at['mediangrossrent']/instances)
    row.at['medianhhincome'] = (row.at['medianhhincome']/instances)
    row.at['gini'] = (row.at['gini']/instances)
    row.at['year'] = (row.at['year']/instances)

In [66]:
for index_label, row_series in weighted_borobox.iterrows():
       # print(index_label)
       # if indexNum > 10:
       #        break
       totalPop = row_series.at['totpop']
       weighted_borobox.loc[index_label,'pctmale'] = ((row_series.at['males']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pct18under'] = ((row_series.at['pop18under']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pct18_34'] = ((row_series.at['pop18_34']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pct35_64'] = ((row_series.at['pop35_64']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pct65plus'] = ((row_series.at['pop65plus']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pcthispanic'] = ((row_series.at['pophispanic']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctnhwhite'] = ((row_series.at['popnhwhite']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctnhblack'] = ((row_series.at['popnhblack']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctnhaian'] = ((row_series.at['popnhaian']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctnhasian'] = ((row_series.at['popnhasian']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctnhhawainpi'] = ((row_series.at['popnhhawainpi']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctnhother'] = ((row_series.at['popnhwhite']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctnhtwoplus'] = ((row_series.at['popnhtwoplus']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctnhaian'] = ((row_series.at['popnhaian']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctownerocch'] = (row_series.at['housingoccupied'] / row_series.at['housing'])*100
       weighted_borobox.loc[index_label,'pctrentocch'] = (row_series.at['renterhousing'] / row_series.at['housing'])*100
       weighted_borobox.loc[index_label, 'crowding']= ((row_series.at['oc_1_onehalf']+row_series.at['oc_onehalf_2']+row_series.at['oc_2plus']) / row_series.at['housing'])*100
       weighted_borobox.loc[index_label, 'crowdingrenter']= ((row_series.at['roc_1_onehalf']+row_series.at['roc_onehalf_2']+row_series.at['roc_2plus']) / row_series.at['renterhousing'])*100
       weighted_borobox.loc[index_label,'pctsamehouse']= (row_series.at['popsamehouse'] / row_series.at['poprenterhu'])*100
       weighted_borobox.loc[index_label,'pctmvdsamecounty']= (row_series.at['popmvdsamecounty'] / row_series.at['poprenterhu'])*100
       weighted_borobox.loc[index_label,'pctmvdsdiffcounty']= (row_series.at['popmvddiffcounty'] / row_series.at['poprenterhu'])*100
       weighted_borobox.loc[index_label,'pctmvdsdiffstate']= (row_series.at['popmvdstate'] / row_series.at['poprenterhu'])*100
       weighted_borobox.loc[index_label,'pctmvdabroad']= (row_series.at['popmvdabroad'] / row_series.at['poprenterhu'])*100
       weighted_borobox.loc[index_label,'pctpoprenterhh']= (row_series.at['poprenterhu'])/(row_series.at['pop1yrplusinhh'])


C:\Users\Jennah\AppData\Local\Temp/ipykernel_14340/970384612.py:23: RuntimeWarning: invalid value encountered in double_scalars
  weighted_borobox.loc[index_label, 'crowdingrenter']= ((row_series.at['roc_1_onehalf']+row_series.at['roc_onehalf_2']+row_series.at['roc_2plus']) / row_series.at['renterhousing'])*100
C:\Users\Jennah\AppData\Local\Temp/ipykernel_14340/970384612.py:24: RuntimeWarning: invalid value encountered in double_scalars
  weighted_borobox.loc[index_label,'pctsamehouse']= (row_series.at['popsamehouse'] / row_series.at['poprenterhu'])*100
C:\Users\Jennah\AppData\Local\Temp/ipykernel_14340/970384612.py:25: RuntimeWarning: invalid value encountered in double_scalars
  weighted_borobox.loc[index_label,'pctmvdsamecounty']= (row_series.at['popmvdsamecounty'] / row_series.at['poprenterhu'])*100
C:\Users\Jennah\AppData\Local\Temp/ipykernel_14340/970384612.py:26: RuntimeWarning: invalid value encountered in double_scalars
  weighted_borobox.loc[index_label,'pctmvdsdiffcounty']= 

In [67]:
median_rent = weighted_borobox["mediangrossrent"].median()
median_income = weighted_borobox["medianhhincome"].median()
median_year = weighted_borobox["medianyearbuilt"].median()
median_gini = weighted_borobox["gini"].median()

m = weighted_borobox.loc[weighted_borobox["mediangrossrent"].isna()]
for index_label, row_series in m.iterrows():
    weighted_borobox.loc[index_label,'mediangrossrent'] = median_rent
m2 = weighted_borobox.loc[weighted_borobox["medianhhincome"].isna()]
for index_label, row_series in m2.iterrows():
    weighted_borobox.loc[index_label,'medianhhincome'] = median_income
m3 = weighted_borobox.loc[weighted_borobox["medianyearbuilt"].isna()]
for index_label, row_series in m3.iterrows():
    weighted_borobox.loc[index_label,'medianyearbuilt'] = median_year
m4 = weighted_borobox.loc[weighted_borobox["gini"].isna()]
for index_label, row_series in m3.iterrows():
    weighted_borobox.loc[index_label,'gini'] = median_gini

In [68]:
weighted_borobox.isna().sum(axis = 0).sort_values(ascending=False).head(15)

borobox             0
pct18_34            0
pctmale             0
popmvdabroad        0
popmvdstate         0
popmvddiffcounty    0
popmvdsamecounty    0
popsamehouse        0
poprenterhu         0
pop1yrplusinhh      0
hhfnofuel           0
hhfotherfuel        0
hhfsolarenergy      0
hhfcoalwood         0
hhfoilkerosene      0
dtype: int64

In [69]:
weighted_borobox.to_csv('weighted_borobox.csv')

In [70]:
# load version from google drive

In [71]:
## load weighted census data
wgt_census = drive.CreateFile({'id': '1H8yM7UH14YdMqi5ASjHMd_dwA4C7-lKe'})
wgt_census.GetContentFile('wgt_census.csv')  

df_wgt_census= pd.read_csv('wgt_census.csv').drop(["Unnamed: 0"], axis = 1)
df_wgt_census.head()

,borobox,count,totpop,popdens,areasqmile,males,females,pop18under,pop18_34,pop35_64,...,pctnhother,pctnhtwoplus,pctownerocch,pctrentocch,pctsamehouse,pctmvdsamecounty,pctmvdsdiffcounty,pctmvdsdiffstate,pctmvdabroad,pctpoprenterhh
0,B0020,1,192.829384,1475.321374,0.007124,94.180095,98.649289,20.819905,81.208531,67.310427,...,54.304813,2.299465,50.742574,49.257426,79.039301,4.279476,13.100437,3.580786,0.0,0.432729
1,B0023,2,414.105781,1448.498316,0.017049,212.526422,201.579359,70.760868,151.051374,168.213951,...,54.304813,2.299465,50.742574,49.257426,79.039301,4.279476,13.100437,3.580786,0.0,0.432729
2,B0024,1,706.381766,4940.829858,0.018737,347.293447,359.088319,104.843305,257.259259,266.170940,...,54.304813,2.299465,50.742574,49.257426,79.039301,4.279476,13.100437,3.580786,0.0,0.432729
3,B0025,1,61.424501,429.637379,0.001629,30.199430,31.225071,9.116809,22.370370,23.145299,...,54.304813,2.299465,50.742574,49.257426,79.039301,4.279476,13.100437,3.580786,0.0,0.432729
4,B0027,1,1335.982906,9344.612991,0.035437,656.837607,679.145299,198.290598,486.555556,503.410256,...,54.304813,2.299465,50.742574,49.257426,79.039301,4.279476,13.100437,3.580786,0.0,0.432729


In [72]:
test= pd.merge(df_wgt_census.loc[:, ['totpop', 'borobox']], weighted_borobox.loc[:, ['totpop', 'borobox']], on = 'borobox')

In [78]:
for c in [x for x in df_wgt_census.columns if x != 'borobox']:
    test= pd.merge(df_wgt_census.loc[:, [c, 'borobox']], weighted_borobox.loc[:, [c, 'borobox']], on = 'borobox')
    test['diff']= test[c+"_x"] - test[c+"_y"]
    if (test['diff'] > 1e-4).any().sum() > 0:
        print(test.loc[test[c+"_x"] != test[c+"_y"], :])
        print(c)
        print(test.loc[test['diff'] > 1e-4, :].shape[0])

         gini_x borobox    gini_y      diff
0      0.026570   B0020  0.038267 -0.011697
1      0.010461   B0023  0.038267 -0.027807
2      0.058031   B0024  0.038267  0.019764
3      0.005046   B0025  0.038267 -0.033221
4      0.109754   B0027  0.038267  0.071487
...         ...     ...       ...       ...
10859  0.066514   X4978  0.038267  0.028247
10860  0.015199   X4981  0.038267 -0.023069
10861  0.023926   X4982  0.038267 -0.014341
10862  0.019802   X4985  0.038267 -0.018465
10863  0.052517   X4986  0.038267  0.014249

[10864 rows x 4 columns]
gini
5419
       pctmale_x borobox  pctmale_y      diff
0       48.28877   B0020   36.75093  11.53784
1       48.28877   B0023   36.75093  11.53784
2       48.28877   B0024   36.75093  11.53784
3       48.28877   B0025   36.75093  11.53784
4       48.28877   B0027   36.75093  11.53784
...          ...     ...        ...       ...
10859   48.28877   X4978   36.75093  11.53784
10860   48.28877   X4981   36.75093  11.53784
10861   48.28877   X49